In [0]:
%pip install openpyxl
import pandas as pd
import openpyxl
from pyspark.sql import SparkSession
import os
import numpy as np

In [0]:

# Path al archivo de interés 2025-2026.xlsx
xlsx_path = os.path.abspath(os.path.join("..", "source_files", "2025-2026.xlsx"))

# Cargar como dataframe
pdf = pd.read_excel(
    xlsx_path,
    header=None,
    dtype=str,
    na_filter=False
)


# Obtener el # de fila con las palabras ['2025', 'Cantidad Solicitada', 'Mínimo']
place_holders = ['2025', 'Cantidad Solicitada', 'Mínimo']
# Obtener headers para la tabla Bronze
# 1. Obtenemos el índice basado para cada place holder
_p1 = pdf.index[pdf.iloc[:, 8] == place_holders[0]]
_p2 = pdf.index[pdf.iloc[:, 8] == place_holders[1]]
_p3 = pdf.index[pdf.iloc[:, 8] == place_holders[2]]

# 2. Extramos filas, reemplazando "" por nan
header_p1 = pdf.iloc[_p1].replace("", pd.NA)
header_p2 = pdf.iloc[_p2].replace("", pd.NA)
header_p3 = pdf.iloc[_p3].replace("", pd.NA)


# 3. Llenamos filas faltantes en el rango de columnas 8-11 y 16 a 20 de la fila 2
header_p2.iloc[:, 5:12] = header_p2.iloc[:, 5:12].ffill(axis=1)
header_p2.iloc[:, 16:] = header_p2.iloc[:, 16:].ffill(axis=1)

# 4. Lleamos las Llenamos filas faltantes en el rango de columnas 8-12 y 16 al final de la fila 1
header_p1.iloc[:, 8:12] = header_p1.iloc[:, 8:12].ffill(axis=1)
header_p1.iloc[:, 16:] = header_p1.iloc[:, 16:].ffill(axis=1)

# 5. Combinamos los dataframes y transponemos, para después combinarlos, la columna header_p1 con los años va al final.
df_headers = pd.concat([header_p3.T, header_p2.T, header_p1.T], axis=1)
headers_raw = df_headers.apply(
    lambda row: ' '.join(row.dropna().astype(str)) if row.notna().any() else np.nan,
    axis=1
)

# 6. Limpiamos 

vocales = str.maketrans({'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u'})

headers = (
    headers_raw  
    .astype(str)                            # ensure strings
    .str.strip()                            # trim whitespace
    .str.lower()                            # lowercase
    .str.replace("cantidad solicitada", "", regex=True)
    .str.replace(r"[\s+.]+", "_", regex=True)
    .str.translate(vocales)
)


# 7. Convertimos a lista
headers_clean = headers.tolist()
print(f"✅ Headers extraídos exitosamente: {headers_clean[0:3]}...")

In [0]:
# Reemplazamos valores vacíos y no relevantes de la primer columna por nan

pdf.iloc[:, 0] = pdf.iloc[:, 0].replace({'No. Partida': np.nan, '': np.nan, '.': np.nan})

# Retiramos los nan basados en la primera columna

data_pdf = pdf.dropna(subset=[pdf.columns[0]])

# Creamos el dataframe de spark
spark_df = spark.createDataFrame(data_pdf) \
                 .toDF(*headers_clean)

# Guardamos la tabla Delta
table = "workspace.default.bronze_2025_2026"
spark_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(table)

print(f"✅ Tabla e ingestión completada: {table}")